In [1]:
from util import *
import pandas as pd 
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)
#Use LSTM to predict
import gzip
import os
import pickle
import requests
import time
import re
os.environ['KERAS_BACKEND']='cntk'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
vizwiz_features_train_color = pd.read_csv('azure_features_images/data/vizwiz_train_color_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                    quotechar='"', error_bad_lines=False)
vizwiz_features_train_text = pd.read_csv('azure_features_images/data/vizwiz_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_features_val_color = pd.read_csv('azure_features_images/data/vizwiz_val_color_recognition.csv',
                                  delimiter=';', engine='python',
                                  dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                  quotechar='"', error_bad_lines=False)
vizwiz_features_val_text = pd.read_csv('azure_features_images/data/vizwiz_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vqa_features_train_color = pd.read_csv('azure_features_images/data/vqa_train_color_recognition.csv',
                                 delimiter=';', engine='python', 
                                 dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                 quotechar='"', error_bad_lines=False)
vqa_features_train_text = pd.read_csv('azure_features_images/data/vqa_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)
vqa_features_val_color = pd.read_csv('azure_features_images/data/vqa_val_color_recognition.csv',
                               delimiter=';', engine='python',
                               dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                               quotechar='"', error_bad_lines=False)
vqa_features_val_text = pd.read_csv('azure_features_images/data/vqa_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_targets_train = pd.read_csv('../vizwiz_skill_typ_train.csv',
                                   delimiter=',', quotechar='"',
                                   engine='python', error_bad_lines=False)
vizwiz_targets_val = pd.read_csv('../vizwiz_skill_typ_val.csv',
                                 delimiter=',', quotechar='"', engine='python', error_bad_lines=False)
vqa_targets_train = pd.read_csv('../vqa_skill_typ_train.csv',
                               engine='python', quotechar='"', error_bad_lines=False)
vqa_targets_val = pd.read_csv('../vqa_skill_typ_val.csv',
                               engine='python', quotechar='"', error_bad_lines=False)

In [7]:
vizwiz_features_train_text.head(2)

,qid,question,ocr_text,handwritten_text
0,VizWiz_train_000000000000.jpg,What's the name of this product?,"['b', 'sil', 'leaves', '0.62', 'oz', '(170)']",['NET WT O. 62 02 ( 179)']
1,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,[],['^TAKE Three 1^']


In [16]:
def join_feature_target(feature_df_text, feature_df_color, target_df):
    
    feature_text = copy.deepcopy(feature_df_text)
    feature_color = copy.deepcopy(feature_df_color)
    target = copy.deepcopy(target_df)
    
    feature_text.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_text.set_index('QID', inplace=True)
    
    feature_color.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_color.set_index('QID', inplace=True)
    
    features = feature_text.join(feature_color, on='QID', how='outer', lsuffix='question')
    
    target = target[['QID', 'IMG', 'QSN', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH']]
    target.set_index('QID', inplace=True)
    target = target.astype(dtype=str)
    
    df = target.join(features, on='QID', how='inner')
    df['descriptions'].astype(list)
    
    return df

In [17]:
vizwiz_train = join_feature_target(vizwiz_features_train_text, 
                                   vizwiz_features_train_color,
                                   vizwiz_targets_train)

In [6]:
vizwiz_train.columns

Index(['IMG', 'QSN', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH', 'question',
       'descriptions', 'tags', 'dominant_colors'],
      dtype='object')

In [25]:
vizwiz_train['descriptions'][0].split(":")[1]

" 'a bottle of wine on a table', 'confidence'"

In [36]:
train_questions = vizwiz_train['QSN'].values
train_

In [ ]:
# tokenize, create seqs, pad
tok = Tokenizer(num_words=VOCAB_SIZE, lower=True, split=" ")
tok.fit_on_texts(ques_train)
train_seq = tok.texts_to_sequences(ques_train)
train_seq = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq = tok.texts_to_sequences(ques_val)
val_seq = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)